In [1]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Mecab
from tqdm import tqdm

In [2]:
import os

In [3]:
os.listdir('../data/')

['df_1.csv', 'nb_table.csv', 'member_mate_2206.csv']

# 5,000개만 test

In [37]:
data = pd.read_csv('../data/df_1.csv')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
df = data[['inqry_conts','mem_sex']]
display(df.head(3))

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,inqry_conts,mem_sex
0,노블레스건은 어떻게 진행되는건가요,f
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요ㅎㅎ\n중년...,f
2,이사람이 연예인 홍일권 이예요ㅎㅎ,f


### 1차 전처리 : 이모티콘, 단모음/단자음, 특문 제거

In [39]:
def basic_preprocessing(df,col):
    df.drop(df[df[col].isnull()].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print('Isnull?', df[col].isnull().sum())
    
    # 공백 제거
    df[col] = df[col].apply(lambda x: x.strip())
    
    # 특수문자, 단모음/단자음 제거
    df[col] = df[col].apply(lambda x: re.sub(r'[\n\r]', ' ',x))
    df[col] = df[col].apply(lambda x: re.sub('[ㄱ-ㅎㅏ-ㅣ]+',' ',x))
    df[col] = df[col].apply(lambda x: re.sub('[a-zA-Z]',' ',x))
    df[col] = df[col].apply(lambda x: re.sub('[-=+,#/&\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]',' ',x))    
    
    # utf-8 encode
    df[col] = df[col].apply(lambda x: x.encode('utf-8','ignore').decode('utf-8'))
    
    # 이모티콘 (이모지) 제거
    def del_emoji (text):
        only_BMP_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                           "]+", flags=re.UNICODE)
        return only_BMP_pattern.sub(r'', text)
    
    df[col] = df[col].apply(del_emoji)

    return df

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:13: DeprecationWarning: invalid escape sequence \?
<>:13: DeprecationWarning: invalid escape sequence \?
<>:13: DeprecationWarning: invalid escape sequence \?
<ipython-input-39-de8db0b3f756>:13: DeprecationWarning: invalid escape sequence \?
  df[col] = df[col].apply(lambda x: re.sub('[-=+,#/&\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]',' ',x))


In [40]:
pre_df = basic_preprocessing(df, 'inqry_conts')
display(pre_df.head(3))

Isnull? 0


/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

,inqry_conts,mem_sex
0,노블레스건은 어떻게 진행되는건가요,f
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f
2,이사람이 연예인 홍일권 이예요,f


In [41]:
test = pre_df.iloc[:10]
test['right'] = ''
display(test)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,inqry_conts,mem_sex,right
0,노블레스건은 어떻게 진행되는건가요,f,
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f,
2,이사람이 연예인 홍일권 이예요,f,
3,닉네임 및 나이를 계속 바꿔가며 가입탈퇴를 반복하는 사기꾼 같아서 신고합니다 확인후...,f,
4,업데이트하라고하는데 업뎃이 안되요,m,
5,구매완료성공했습니다,f,
6,가려진정보본기는 또다시결제를해야되는건가요,f,
7,상대회원은 재혼 대구 48세 레터 대화명 입니다 확인좀 부탁드려요,f,
8,3월 21일자 절 관심등록한 회원이 이튿날 프로필삭제를 한후에 며칠지나서 다시 재가...,f,
9,안녕하세요 어제 여성분과 메세지 주고받던 메세시창이 갑자기 사라져 버렸네요 그리고...,m,


### 2차 전처리 : 맞춤법 교정

In [42]:
from hanspell import spell_checker

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
def get_right_spell(df, col):
    
    for idx, val in enumerate(df[col]):
        col_idx = idx
        
        try:
            result = spell_checker.check(val)
            right_spell = result.as_dict()['checked']
            df.loc[col_idx, 'right'] = right_spell
            
        except:
            df.loc[col_idx, 'right'] = 'error'
        
    return df

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
get_right_spell(test, 'inqry_conts')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,inqry_conts,mem_sex,right
0,노블레스건은 어떻게 진행되는건가요,f,노블레스건은 어떻게 진행되는 건가요
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f,이 회원 지난번에도 연예인 사진 도용하더니 탈퇴하고 또 연예인 사진 올렸네요 중년...
2,이사람이 연예인 홍일권 이예요,f,이 사람이 연예인 홍일권 이예요
3,닉네임 및 나이를 계속 바꿔가며 가입탈퇴를 반복하는 사기꾼 같아서 신고합니다 확인후...,f,닉네임 및 나이를 계속 바꿔가며 가입 탈퇴를 반복하는 사기꾼 같아서 신고합니다 확인...
4,업데이트하라고하는데 업뎃이 안되요,m,업데이트하라고 하는데 업데이트가 안돼요
5,구매완료성공했습니다,f,구매완료 성공했습니다
6,가려진정보본기는 또다시결제를해야되는건가요,f,가려진 정보 본지는 또다시 결제를 해야 되는 건가요
7,상대회원은 재혼 대구 48세 레터 대화명 입니다 확인좀 부탁드려요,f,상대 회원은 재혼 대구 48세 레터 대화명입니다 확인 좀 부탁드려요
8,3월 21일자 절 관심등록한 회원이 이튿날 프로필삭제를 한후에 며칠지나서 다시 재가...,f,3월 21일 자 절 관심등록한 회원이 이튿날 프로필 삭제를 한 후에 며칠 지나서 다...
9,안녕하세요 어제 여성분과 메세지 주고받던 메세시창이 갑자기 사라져 버렸네요 그리고...,m,안녕하세요 어제 여성분과 메시지 주고받던 메세 시창이 갑자기 사라져 버렸네요 그리...


In [45]:
%%time

get_right_spell(pre_df, 'inqry_conts')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

CPU times: user 47.2 s, sys: 515 ms, total: 47.7 s
Wall time: 6min 31s


,inqry_conts,mem_sex,right
0,노블레스건은 어떻게 진행되는건가요,f,노블레스건은 어떻게 진행되는 건가요
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f,이 회원 지난번에도 연예인 사진 도용하더니 탈퇴하고 또 연예인 사진 올렸네요 중년...
2,이사람이 연예인 홍일권 이예요,f,이 사람이 연예인 홍일권 이예요
3,닉네임 및 나이를 계속 바꿔가며 가입탈퇴를 반복하는 사기꾼 같아서 신고합니다 확인후...,f,닉네임 및 나이를 계속 바꿔가며 가입 탈퇴를 반복하는 사기꾼 같아서 신고합니다 확인...
4,업데이트하라고하는데 업뎃이 안되요,m,업데이트하라고 하는데 업데이트가 안돼요
...,...,...,...
4995,안녕 하세요 무슨 콜 센타가 지금까지 전화를 안 받습니까 회사 운영는하나요,m,안녕하세요 무슨 콜센터가 지금까지 전화를 안 받습니까 회사 운영은 하나요
4996,대화명 수정하다 잘못 수정되어 대화명 변경부탁드립니다 1 화려한 날 2 꿈꾸며...,f,대화명 수정하다 잘못 수정되어 대화명 변경 부탁드립니다 1 화려한 날 2 꿈꾸...
4997,연상과 나이차로 메시지를 받을수 없는 획일적으로 차단하는건 누가봐도 이해가되지 않...,m,연상과 나이차로 메시지를 받을 수 없는 획일적으로 차단하는 건 누가 봐도 이해가 ...
4998,애니13분에게 메시지를 보내려고 하는데 안보내집니다 나이 차이가 있어 안된다는데 요...,m,애니 13분에게 메시지를 보내려고 하는데 안 보내집니다 나이 차이가 있어 안된다는데...


In [46]:
f_df = pre_df.loc[pre_df['mem_sex']=='f'].reset_index(drop=True)
m_df = pre_df.loc[pre_df['mem_sex']=='m'].reset_index(drop=True)

display(f_df.head(2))
display(m_df.head(2))

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,inqry_conts,mem_sex,right
0,노블레스건은 어떻게 진행되는건가요,f,노블레스건은 어떻게 진행되는 건가요
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f,이 회원 지난번에도 연예인 사진 도용하더니 탈퇴하고 또 연예인 사진 올렸네요 중년...


,inqry_conts,mem_sex,right
0,업데이트하라고하는데 업뎃이 안되요,m,업데이트하라고 하는데 업데이트가 안돼요
1,안녕하세요 어제 여성분과 메세지 주고받던 메세시창이 갑자기 사라져 버렸네요 그리고...,m,안녕하세요 어제 여성분과 메시지 주고받던 메세 시창이 갑자기 사라져 버렸네요 그리...


### 토크나이징

In [47]:

def get_nouns(x):
    tokenizer = Mecab()
    tagged = tokenizer.pos(x)
    nouns = [s for s,t in tagged if t in ['NNG','NNP', 'VA', 'XR'] and len(s) >1]
#   ','.join(n for n in nouns)
    return nouns

pre_df['noun'] = pre_df['right'].apply(get_nouns)
display(pre_df.head(3))

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,inqry_conts,mem_sex,right,noun
0,노블레스건은 어떻게 진행되는건가요,f,노블레스건은 어떻게 진행되는 건가요,"[노블레스, 진행]"
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f,이 회원 지난번에도 연예인 사진 도용하더니 탈퇴하고 또 연예인 사진 올렸네요 중년...,"[회원, 지난번, 연예인, 사진, 용하, 탈퇴, 연예인, 사진, 중년, 배우, 홍일..."
2,이사람이 연예인 홍일권 이예요,f,이 사람이 연예인 홍일권 이예요,"[사람, 연예인, 홍일]"


### LDA 토픽 모델링

In [48]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
def get_lst(df, col):
    
    lst = []
    for idx, row in enumerate(df[col]):
        lst.append(row)
        
    return lst

processed_data = get_lst(pre_df, 'noun')
processed_data

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['노블레스', '진행'],
 ['회원',
  '지난번',
  '연예인',
  '사진',
  '용하',
  '탈퇴',
  '연예인',
  '사진',
  '중년',
  '배우',
  '홍일',
  '사진',
  '신고'],
 ['사람', '연예인', '홍일'],
 ['닉네임', '나이', '가입', '탈퇴', '반복', '사기', '신고', '확인', '조치'],
 ['업데이트', '업데이트'],
 ['구매', '완료', '성공'],
 ['정보', '본지', '결제'],
 ['상대', '회원', '재혼', '대구', '레터', '대화', '확인', '부탁'],
 ['관심',
  '등록',
  '회원',
  '이튿날',
  '프로필',
  '삭제',
  '며칠',
  '가입',
  '소식란',
  '관심',
  '등록',
  '회원',
  '명단',
  '프로필',
  '삭제',
  '관심',
  '등록',
  '등록',
  '날짜',
  '관심',
  '등록',
  '오류',
  '상대방',
  '대화',
  '프로필',
  '작성',
  '프로필',
  '삭제',
  '차단',
  '현상',
  '궁금'],
 ['안녕', '여성', '분과', '메시지', '메세', '시창', '프로필', '여성', '수상', '주작', '냄새'],
 ['관심',
  '등록',
  '사람',
  '프로필',
  '삭제',
  '가입',
  '관심',
  '등록',
  '관심',
  '등록',
  '상태',
  '관심',
  '해제',
  '자유',
  '설정',
  '가능'],
 ['사기', '가입', '정지', '이번', '사진', '프로필', '확인', '비슷'],
 ['사진', '카드', '터치', '해결'],
 ['노블레스',
  '마음',
  '최선',
  '인증',
  '생각',
  '직장',
  '정보',
  '인증',
  '문제',
  '휴무',
  '재직',
  '증명서',
  '어렵',
  '현장',
  '명함',
  '월급',
  '명세서',
  '인증',

In [51]:
dicts = corpora.Dictionary(processed_data)
dicts

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 13:34:51,321 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-06-28 13:34:51,363 : INFO : built Dictionary<3599 unique tokens: ['노블레스', '진행', '배우', '사진', '신고']...> from 5000 documents (total 35366 corpus positions)
2022-06-28 13:34:51,364 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<3599 unique tokens: ['노블레스', '진행', '배우', '사진', '신고']...> from 5000 documents (total 35366 corpus positions)", 'datetime': '2022-06-28T13:34:51.364312', 'gensim': '4.2.0', 'python': '3.7.11 (default, Jul 27 2021, 14:32:16) \n[GCC 7.5.0]', 'platform': 'Linux-4.15.0-180-generic-x86_64-with-debian-

In [52]:
corpus = [dicts.doc2bow(text) for text in processed_data]
corpus

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1)],
 [(2, 1),
  (3, 3),
  (4, 1),
  (5, 2),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1)],
 [(5, 1), (10, 1), (12, 1)],
 [(4, 1),
  (9, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1)],
 [(20, 2)],
 [(21, 1), (22, 1), (23, 1)],
 [(24, 1), (25, 1), (26, 1)],
 [(11, 1), (19, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)],
 [(11, 2),
  (13, 1),
  (28, 1),
  (33, 4),
  (34, 1),
  (35, 1),
  (36, 5),
  (37, 1),
  (38, 1),
  (39, 3),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 4),
  (47, 1)],
 [(46, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1)],
 [(12, 1),
  (13, 1),
  (33, 4),
  (36, 3),
  (39, 1),
  (46, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(3, 1), (13, 1), (17, 1), (19, 1), (46, 1), (62, 1), (63, 1), (64, 1)],
 [(3, 1), (65, 1), (66, 1), (67, 1)],
 [(0, 1),
  (26, 1),
  (30, 1),
  (68, 1),
  (69, 1),
  (70,

#### 모델링 
조정이 필요한 할 변수 :num_topics, chunksize, passes, interations, eval_every

- num_topics: 생성될 토픽의 개수
- chunksize: 한번의 트레이닝에 처리될 문서의 개수
-  passes: 전체 코퍼스 트레이닝 횟수
- interations: 문서 당 반복 횟수

In [53]:
%%time
# 모델링

num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dicts[0]
id2word = dicts.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 13:34:51,599 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2022-06-28 13:34:51,600 : INFO : using serial LDA version on this node
2022-06-28 13:34:51,602 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 5000 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-06-28 13:34:51,602 : INFO : PROGRESS: pass 0, at document #2000/5000
2022-06-28 13:34:52,239 : INFO : optimized alpha [0.14291432, 0.16328461, 0.14890628, 0.15438029, 0.1574461

CPU times: user 13.8 s, sys: 1.16 s, total: 14.9 s
Wall time: 13.7 s


In [54]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 13:35:05,296 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-06-28 13:35:05,299 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-06-28 13:35:05,302 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-06-28 13:35:05,304 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-06-28 13:35:05,307 : INFO : CorpusAccumulator accumulated stats from 5000 documents


Average topic coherence: -3.4854.
[([(0.039686155, '인증'),
   (0.031098463, '사진'),
   (0.025501486, '나이'),
   (0.024435917, '본인'),
   (0.02124655, '메시지'),
   (0.01989863, '회원'),
   (0.018205954, '확인'),
   (0.017415289, '사람'),
   (0.012366494, '가입'),
   (0.0113586, '프로필'),
   (0.010827638, '문자'),
   (0.009668342, '대화'),
   (0.009227129, '여성'),
   (0.008599469, '상대'),
   (0.0084565, '생각'),
   (0.008394814, '연락'),
   (0.008228228, '신고'),
   (0.007966479, '부탁'),
   (0.007824231, '이상'),
   (0.007506951, '쪽지')],
  -2.7215910757608754),
 ([(0.06284502, '결제'),
   (0.05314091, '코인'),
   (0.039540794, '구매'),
   (0.039013386, '메시지'),
   (0.03278924, '결재'),
   (0.03191234, '충전'),
   (0.031854086, '사용'),
   (0.024567524, '입금'),
   (0.024331136, '번호'),
   (0.0203888, '탈퇴'),
   (0.019887654, '부탁'),
   (0.018470604, '문자'),
   (0.018443296, '이용'),
   (0.01766247, '확인'),
   (0.015707713, '환불'),
   (0.015564042, '전화'),
   (0.013936023, '금액'),
   (0.013846508, '취소'),
   (0.012876512, '아이템'),
   (0.01207150

#### 시각화

In [56]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
lda_visualization = gensimvis.prepare(model, corpus, dicts, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_5000.html')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


# 300,000개  test

In [57]:
G = pd.read_csv('../data/g.csv')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [58]:
g_df = G[['inqry_conts','mem_sex']]
display(g_df.head(3))


/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,inqry_conts,mem_sex
0,노블레스건은 어떻게 진행되는건가요,f
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요ㅎㅎ\n중년...,f
2,이사람이 연예인 홍일권 이예요ㅎㅎ,f


In [59]:
g_pre_df = basic_preprocessing(g_df, 'inqry_conts')
display(g_pre_df.head(3))

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Isnull? 0


/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,inqry_conts,mem_sex
0,노블레스건은 어떻게 진행되는건가요,f
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f
2,이사람이 연예인 홍일권 이예요,f


In [60]:
%%time

get_right_spell(g_pre_df, 'inqry_conts')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

CPU times: user 6min 35s, sys: 3.76 s, total: 6min 39s
Wall time: 47min 53s


,inqry_conts,mem_sex,right
0,노블레스건은 어떻게 진행되는건가요,f,노블레스건은 어떻게 진행되는 건가요
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f,이 회원 지난번에도 연예인 사진 도용하더니 탈퇴하고 또 연예인 사진 올렸네요 중년...
2,이사람이 연예인 홍일권 이예요,f,이 사람이 연예인 홍일권 이예요
3,닉네임 및 나이를 계속 바꿔가며 가입탈퇴를 반복하는 사기꾼 같아서 신고합니다 확인후...,f,닉네임 및 나이를 계속 바꿔가며 가입 탈퇴를 반복하는 사기꾼 같아서 신고합니다 확인...
4,업데이트하라고하는데 업뎃이 안되요,m,업데이트하라고 하는데 업데이트가 안돼요
...,...,...,...
36555,로그인 어떤 유형의 사람일까요 혹시 영화 속 외계인은 아니죠 그대는 어느 별...,m,로그인 어떤 유형의 사람일까요 혹시 영화 속 외계인은 아니죠 그대는 어느 별...
36556,비밀번호가 기억안날땐 어떻게 찾을수있나요 비밀번호 넣은기억이 없는데,f,비밀번호가 기억 안날 땐 어떻게 찾을 수 있나요 비밀번호 넣은 기억이 없는데
36557,탈퇴 하고 재가입 하려고 하니까 탈퇴 하려니까 네이버 간편해서 비밀번호를 누르라고 ...,m,탈퇴하고 재가입 하려고 하니까 탈퇴하려니까 네이버 간편해서 비밀번호를 누르라고 하는...
36558,그분과 쪽지 나눌수있도록해주세요 명예홰손으로 고소할겁니다 증거다있습니다,f,그분과 쪽지 나눌 수 있도록 해주세요 명예훼손으로 고소할 겁니다 증거다 있습니다


In [61]:
g_pre_df['noun'] = g_pre_df['right'].apply(get_nouns)
display(g_pre_df.head(3))

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,inqry_conts,mem_sex,right,noun
0,노블레스건은 어떻게 진행되는건가요,f,노블레스건은 어떻게 진행되는 건가요,"[노블레스, 진행]"
1,이 회원 지난번에도 연예인사진 도용하더니 탈퇴하고 또 연예인사진 올렸네요 중년배우...,f,이 회원 지난번에도 연예인 사진 도용하더니 탈퇴하고 또 연예인 사진 올렸네요 중년...,"[회원, 지난번, 연예인, 사진, 용하, 탈퇴, 연예인, 사진, 중년, 배우, 홍일..."
2,이사람이 연예인 홍일권 이예요,f,이 사람이 연예인 홍일권 이예요,"[사람, 연예인, 홍일]"


In [62]:
g_processed_data = get_lst(g_pre_df, 'noun')
g_processed_data

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['노블레스', '진행'],
 ['회원',
  '지난번',
  '연예인',
  '사진',
  '용하',
  '탈퇴',
  '연예인',
  '사진',
  '중년',
  '배우',
  '홍일',
  '사진',
  '신고'],
 ['사람', '연예인', '홍일'],
 ['닉네임', '나이', '가입', '탈퇴', '반복', '사기', '신고', '확인', '조치'],
 ['업데이트', '업데이트'],
 ['구매', '완료', '성공'],
 ['정보', '본지', '결제'],
 ['상대', '회원', '재혼', '대구', '레터', '대화', '확인', '부탁'],
 ['관심',
  '등록',
  '회원',
  '이튿날',
  '프로필',
  '삭제',
  '며칠',
  '가입',
  '소식란',
  '관심',
  '등록',
  '회원',
  '명단',
  '프로필',
  '삭제',
  '관심',
  '등록',
  '등록',
  '날짜',
  '관심',
  '등록',
  '오류',
  '상대방',
  '대화',
  '프로필',
  '작성',
  '프로필',
  '삭제',
  '차단',
  '현상',
  '궁금'],
 ['안녕', '여성', '분과', '메시지', '메세', '시창', '프로필', '여성', '수상', '주작', '냄새'],
 ['관심',
  '등록',
  '사람',
  '프로필',
  '삭제',
  '가입',
  '관심',
  '등록',
  '관심',
  '등록',
  '상태',
  '관심',
  '해제',
  '자유',
  '설정',
  '가능'],
 ['사기', '가입', '정지', '이번', '사진', '프로필', '확인', '비슷'],
 ['사진', '카드', '터치', '해결'],
 ['노블레스',
  '마음',
  '최선',
  '인증',
  '생각',
  '직장',
  '정보',
  '인증',
  '문제',
  '휴무',
  '재직',
  '증명서',
  '어렵',
  '현장',
  '명함',
  '월급',
  '명세서',
  '인증',

In [63]:
g_dicts = corpora.Dictionary(g_processed_data)
g_dicts

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 14:23:28,649 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-06-28 14:23:28,730 : INFO : adding document #10000 to Dictionary<5336 unique tokens: ['노블레스', '진행', '배우', '사진', '신고']...>
2022-06-28 14:23:28,808 : INFO : adding document #20000 to Dictionary<7633 unique tokens: ['노블레스', '진행', '배우', '사진', '신고']...>
2022-06-28 14:23:28,884 : INFO : adding document #30000 to Dictionary<9599 unique tokens: ['노블레스', '진행', '배우', '사진', '신고']...>
2022-06-28 14:23:28,931 : INFO : built Dictionary<10722 unique tokens: ['노블레스', '진행', '배우', '사진', '신고']...> from 36560 documents (total 239561 corpus position

In [64]:
g_corpus = [g_dicts.doc2bow(text) for text in g_processed_data]
g_corpus

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1)],
 [(2, 1),
  (3, 3),
  (4, 1),
  (5, 2),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1)],
 [(5, 1), (10, 1), (12, 1)],
 [(4, 1),
  (9, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1)],
 [(20, 2)],
 [(21, 1), (22, 1), (23, 1)],
 [(24, 1), (25, 1), (26, 1)],
 [(11, 1), (19, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)],
 [(11, 2),
  (13, 1),
  (28, 1),
  (33, 4),
  (34, 1),
  (35, 1),
  (36, 5),
  (37, 1),
  (38, 1),
  (39, 3),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 4),
  (47, 1)],
 [(46, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1)],
 [(12, 1),
  (13, 1),
  (33, 4),
  (36, 3),
  (39, 1),
  (46, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(3, 1), (13, 1), (17, 1), (19, 1), (46, 1), (62, 1), (63, 1), (64, 1)],
 [(3, 1), (65, 1), (66, 1), (67, 1)],
 [(0, 1),
  (26, 1),
  (30, 1),
  (68, 1),
  (69, 1),
  (70,

In [65]:
%%time
# 모델링

num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = g_dicts[0]
id2word = g_dicts.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 14:23:29,243 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2022-06-28 14:23:29,245 : INFO : using serial LDA version on this node
2022-06-28 14:23:29,250 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 5000 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-06-28 14:23:29,250 : INFO : PROGRESS: pass 0, at document #2000/5000
2022-06-28 14:23:29,894 : INFO : optimized alpha [0.14341775, 0.13376135, 0.16338441, 0.15522392, 0.1704265

CPU times: user 13.9 s, sys: 1.11 s, total: 15 s
Wall time: 13.8 s


In [66]:
g_top_topics = model.top_topics(g_corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
g_avg_topic_coherence = sum([t[1] for t in g_top_topics]) / num_topics
print('Average topic coherence: %.4f.' % g_avg_topic_coherence)

from pprint import pprint
pprint(g_top_topics)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 14:23:43,058 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-06-28 14:23:43,060 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-06-28 14:23:43,063 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-06-28 14:23:43,066 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-06-28 14:23:43,068 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-06-28 14:23:43,071 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-06-28 14:23:43,074 : INFO : CorpusAccumulator accumulated stats from 7000 documents
202

Average topic coherence: -6.1143.
[([(0.06644871, '프로필'),
   (0.0606021, '차단'),
   (0.02154373, '기능'),
   (0.0202889, '메시지'),
   (0.019977292, '열람'),
   (0.019821478, '대화'),
   (0.019391041, '회원'),
   (0.019126642, '상대방'),
   (0.017796014, '신고'),
   (0.014410254, '사람'),
   (0.014129972, '상대'),
   (0.010001756, '내용'),
   (0.009925272, '연락'),
   (0.009207558, '확인'),
   (0.009033748, '허위'),
   (0.008304237, '열람권'),
   (0.008004602, '삭제'),
   (0.0075602364, '부탁'),
   (0.007351757, '관심'),
   (0.0071067074, '전체')],
  -3.0422450904852285),
 ([(0.061743166, '결제'),
   (0.05345405, '코인'),
   (0.04197636, '메시지'),
   (0.039770763, '구매'),
   (0.032983698, '결재'),
   (0.03210044, '충전'),
   (0.03195042, '사용'),
   (0.02474055, '입금'),
   (0.020903718, '문자'),
   (0.019700808, '이용'),
   (0.016341023, '확인'),
   (0.01580011, '환불'),
   (0.015207781, '부탁'),
   (0.014018154, '금액'),
   (0.013927095, '취소'),
   (0.012948868, '아이템'),
   (0.011007199, '관심'),
   (0.010148547, '오류'),
   (0.010004936, '구입'),
   (0.007

In [68]:
lda_visualization = gensimvis.prepare(model, g_corpus, g_dicts, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_30000.html')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 700,000개  test

In [69]:
all_data = pd.read_csv('../data/admin_QA.csv')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
all_df = all_data[['inqry_conts','mem_sex']]

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [72]:
all_pre_df = basic_preprocessing(all_df, 'inqry_conts')
display(all_pre_df.head(3))

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Isnull? 0


/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,inqry_conts,mem_sex
0,앙돼요,m
1,앙돼요2,m
2,,m


In [73]:
%%time

get_right_spell(all_pre_df, 'inqry_conts')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

CPU times: user 15min 49s, sys: 8.61 s, total: 15min 57s
Wall time: 1h 40min 17s


,inqry_conts,mem_sex,right
0,앙돼요,m,안돼요
1,앙돼요2,m,안돼요 2
2,,m,
3,메시지함이 갑자기 텅비워 졌네요 메시지함에 아무것도 나타 나지 않아요,m,메시지 함이 갑자기 텅 비워졌네요 메시지함에 아무것도 나타나지 않아요
4,결제방법이 계좌이체는 안되나요 결제할려니 구글로그인을 해야되는가요 구글 비번...,m,결제방법이 계좌이체는 안 되나요 결제하려니 구글 로그인을 해야 되는가요 구글...
...,...,...,...
73127,1만원결제했는데 회원정보 왜열람이 안될까요,f,1만 원 결제했는데 회원정보 왜 열람이 안될까요
73128,가려진정보본기는 또다시결제를해야되는건가요,f,가려진 정보 본지는 또다시 결제를 해야 되는 건가요
73129,구매완료성공했습니다,f,구매완료 성공했습니다
73130,대화명수정하고 재혼으로 변경해야됩니다,m,대화명 수정하고 재혼으로 변경해야 됩니다


In [74]:
all_pre_df['noun'] = all_pre_df['right'].apply(get_nouns)
display(all_pre_df.head(3))

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,inqry_conts,mem_sex,right,noun
0,앙돼요,m,안돼요,[]
1,앙돼요2,m,안돼요 2,[]
2,,m,,[]


In [92]:
del_idx = all_pre_df.loc[all_pre_df['noun'].str.len()==0].index
all_pre_df = all_pre_df.drop(del_idx)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [93]:
all_pre_df.reset_index(drop=True, inplace=True)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
all_processed_data = get_lst(all_pre_df, 'noun')
all_processed_data

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['메시지', '메시지'],
 ['결제',
  '방법',
  '계좌',
  '이체',
  '결제',
  '구글',
  '로그인',
  '가요',
  '구글',
  '비번',
  '로그인',
  '결제',
  '어렵'],
 ['결제',
  '방법',
  '방법',
  '결제',
  '복잡',
  '인증',
  '인증',
  '어렵',
  '결제',
  '포기',
  '결제',
  '수단',
  '정할'],
 ['사남'],
 ['오려'],
 ['수정'],
 ['차단', '회원'],
 ['탈퇴'],
 ['회원', '탈퇴', '방법'],
 ['탈퇴', '대화', '재즈'],
 ['회원', '적절', '조치'],
 ['안녕', '회원', '정리', '생각', '확인', '처리', '수고'],
 ['안녕', '시크릿', '서비스', '이용', '사진', '공개'],
 ['실수',
  '로그아웃',
  '비밀',
  '번호',
  '로그',
  '변경',
  '오류',
  '임시',
  '번호',
  '로스',
  '비번',
  '변경',
  '기존',
  '비번',
  '입력',
  '번호',
  '입력',
  '기존',
  '비번',
  '입력',
  '기존',
  '비번',
  '삭제',
  '비번',
  '입력',
  '기존',
  '비번',
  '메일',
  '감사',
  '감사'],
 ['문자', '확인', '메시지', '가요'],
 ['상대방', '메시지', '전송이', '궁금', '별도', '결제', '답변', '부탁'],
 ['문의', '테스트'],
 ['문의', '테스트'],
 ['테스트'],
 ['테스트'],
 ['테스트'],
 ['테스트'],
 ['터치'],
 ['터치'],
 ['테스트'],
 ['테스트'],
 ['테스트'],
 ['테스트'],
 ['테스트'],
 ['테스트'],
 ['모지'],
 ['모지'],
 ['테스트'],
 ['테스트'],
 ['재혼', '사이트', '나이', '관리', '정신', '무분별'],
 ['사람',
  '인물',
 

In [95]:
all_dicts = corpora.Dictionary(all_processed_data)
all_dicts

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 16:12:48,510 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-06-28 16:12:48,585 : INFO : adding document #10000 to Dictionary<4651 unique tokens: ['메시지', '가요', '결제', '계좌', '구글']...>
2022-06-28 16:12:48,662 : INFO : adding document #20000 to Dictionary<7539 unique tokens: ['메시지', '가요', '결제', '계좌', '구글']...>
2022-06-28 16:12:48,736 : INFO : adding document #30000 to Dictionary<9434 unique tokens: ['메시지', '가요', '결제', '계좌', '구글']...>
2022-06-28 16:12:48,811 : INFO : adding document #40000 to Dictionary<11046 unique tokens: ['메시지', '가요', '결제', '계좌', '구글']...>
2022-06-28 16:12:48,887 : INFO : a

In [96]:
all_corpus = [all_dicts.doc2bow(text) for text in all_processed_data]
all_corpus

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 2)],
 [(1, 1), (2, 3), (3, 1), (4, 2), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(2, 4), (6, 2), (8, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1)],
 [(15, 1)],
 [(16, 1)],
 [(17, 1)],
 [(18, 1), (19, 1)],
 [(20, 1)],
 [(6, 1), (19, 1), (20, 1)],
 [(20, 1), (21, 1), (22, 1)],
 [(19, 1), (23, 1), (24, 1)],
 [(19, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)],
 [(27, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)],
 [(7, 6),
  (36, 2),
  (37, 4),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 3),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 4)],
 [(0, 1), (1, 1), (30, 1), (50, 1)],
 [(0, 1), (2, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)],
 [(57, 1), (58, 1)],
 [(57, 1), (58, 1)],
 [(58, 1)],
 [(58, 1)],
 [(58, 1)],
 [(58, 1)],
 [(59, 1)],
 [(59, 1)],
 [(58, 1)],
 [(58, 1)],
 [(58, 1)],
 [(58, 1)],
 [(58, 1)],
 [(58, 1)],
 [(60, 1)],
 [(60, 1)],
 [(58, 1)],
 [(58, 1)],
 [(61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 

In [99]:
%%time
# 모델링

num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = all_dicts[0]
id2word = all_dicts.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 16:14:44,676 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2022-06-28 16:14:44,678 : INFO : using serial LDA version on this node
2022-06-28 16:14:44,685 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 5000 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-06-28 16:14:44,685 : INFO : PROGRESS: pass 0, at document #2000/5000
2022-06-28 16:14:45,261 : INFO : optimized alpha [0.15045205, 0.16408817, 0.14840133, 0.15136182, 0.1522454

CPU times: user 13.4 s, sys: 976 ms, total: 14.4 s
Wall time: 13.4 s


In [100]:
all_top_topics = model.top_topics(all_corpus) #, num_words=20)

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-06-28 16:15:02,448 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-06-28 16:15:02,450 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-06-28 16:15:02,452 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-06-28 16:15:02,454 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-06-28 16:15:02,456 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-06-28 16:15:02,457 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-06-28 16:15:02,460 : INFO : CorpusAccumulator accumulated stats from 7000 documents
202

In [101]:
# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in all_top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

Average topic coherence: -12.8760.


/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [102]:
lda_visualization = gensimvis.prepare(model, all_corpus, all_dicts, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'LDA_70000.html')

/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/yeoai/anaconda3/envs/nlp/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
